In [15]:
import os
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu'#very important theano is initialized with cpu

import subprocess
import cPickle
import logging
import sys

import numpy as np
import random
from copy import copy

import blocks
from blocks.bricks import Linear, Softmax, Softplus, NDimensionalSoftmax, BatchNormalizedMLP,\
                          Rectifier, Logistic, Tanh, MLP
from blocks.bricks.recurrent import GatedRecurrent, Fork, LSTM
from blocks.initialization import Constant, IsotropicGaussian, Identity, Uniform
from blocks.bricks.cost import BinaryCrossEntropy, CategoricalCrossEntropy
from blocks.filter import VariableFilter
from blocks.roles import PARAMETER
from blocks.graph import ComputationGraph

import theano
from theano import tensor as T

module_logger = logging.getLogger(__name__)
sys.setrecursionlimit(100000)

In [16]:
def pickleFile(thing2save, file2save2=None, filePath='/work/notebooks/drawModels/', fileName='myModels'):  # pragma: no cover
    if file2save2 is None:
        f = file(filePath+fileName+'.pickle', 'wb')
    else:
        f = file(filePath+file2save2, 'wb')

    cPickle.dump(thing2save, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

def loadFile(filePath):  # pragma: no cover
    file2open = file(filePath, 'rb')
    loadedFile = cPickle.load(file2open)
    file2open.close()
    return loadedFile

In [17]:
def convertgpu2cpu(gpuLoadModelPath, cpuSaveModelPath, cpuModelName = 'cpuVersion'):
    gpumodel = loadFile(gpuLoadModelPath)
    
    ################################################
    # This model graph has to be the EXACT graph used to generate the gpumodel
    ################################################
    
    ######Model graph
    
    X = T.tensor4('inputs')
    Y = T.matrix('targets')
    rnnType = 'gru'

    #data specifics
    dimIn = 257
    maxPackets = 8
    dim = 100
    numClasses = 4

    ###ENCODER
    if rnnType == 'gru':
        rnn = GatedRecurrent(dim=dim)
        dimMultiplier = 2
    else:
        rnn = LSTM(dim=dim)
        dimMultiplier = 4

    fork = Fork(output_names=['linear', 'gates'], input_dim=dimIn, output_dims=[dim, dim * dimMultiplier])

    ###CONTEXT
    if rnnType == 'gru':
        rnnContext = GatedRecurrent(dim=dim)
    else:
        rnnContext = LSTM(dim=dim)

    forkContext = Fork(output_names=['linearContext', 'gatesContext'], input_dim=dim, output_dims=[dim, dim * dimMultiplier])

    forkDec = Fork(output_names=['linearOut', 'gatesOut'], input_dim=dim, output_dims=[dim, dim*dimMultiplier])

    #CLASSIFIER
    bmlp = BatchNormalizedMLP( activations=[Logistic(),Logistic()],dims=[dim, dim, numClasses])

    ####No initialization needed
    
    def onestepEnc(X):
        data1, data2 = fork.apply(X) 

        if rnnType == 'gru':
            hEnc = rnn.apply(data1, data2) 
        else:
            hEnc, _ = rnn.apply(data2)

        return hEnc

    hEnc, _ = theano.scan(onestepEnc, X) #(mini*numPackets, packetLen, 1, hexdictLen)
    hEncReshape = T.reshape(hEnc[:,-1], (-1, maxPackets, 1, dim)) #[:,-1] takes the last rep for each packet
                                                                 #(mini, numPackets, 1, dimReduced)

    def onestepContext(hEncReshape):

        data3, data4 = forkContext.apply(hEncReshape)

        if rnnType == 'gru':
            hContext = rnnContext.apply(data3, data4)
        else:
            hContext, _ = rnnContext.apply(data4)

        return hContext


    hContext, _ = theano.scan(onestepContext, hEncReshape)
    hContextReshape = T.reshape(hContext[:,-1], (-1,dim))

    data5, _ = forkDec.apply(hContextReshape)

    pyx = bmlp.apply(data5)
    softmax = Softmax()
    softoutClass = softmax.apply(pyx)

    cpumodel = theano.function([X], softoutClass, allow_input_downcast=True)
    
    #Now both gpumodel and cpumodel have the same parameters, load cpumode's parameters with gpumodel's
    for wt in range(len(cpumodel.get_shared())):
        cpumodel.get_shared()[wt].set_value(gpumodel.get_shared()[wt].get_value())
    
    pickleFile(cpumodel, filePath=cpuSaveModelPath, fileName=cpuModelName)
    
    return cpumodel

In [18]:
#test
cpumodel = convertgpu2cpu(gpuLoadModelPath = '/data/fs4/home/bradh/outputs/hredClassify8fullpacketsPREDICT61500.pickle', 
                          cpuSaveModelPath = '/data/fs4/home/bradh/outputs/', 
                          cpuModelName = 'cpuVersion')